In [1]:
from sgp4.api import Satrec
from datetime import datetime
from numpy import datetime64

s = "1 42956U 17061B   23296.12655675  .00000536  00000+0  18443-3 0  9996"
t = "2 42956  86.3950  69.7571 0001996  92.0127 268.1297 14.34219595316042"

satellite = Satrec.twoline2rv(s, t)

from astropy import units as u
from astropy.time import Time

start_time_dt = datetime64(datetime.strptime("2023-10-22-21-39-19", "%Y-%m-%d-%H-%M-%S"))
start_time = Time(start_time_dt, format="datetime64")
start_time

<Time object: scale='utc' format='datetime64' value=2023-10-22T21:39:19.000000000>

In [2]:
from poliastro.util import time_range
from astropy import units as u

times = time_range(start_time, spacing=u.Quantity(1, u.s), periods=26)[15:]
times

<Time object: scale='utc' format='datetime64' value=['2023-10-22T21:39:34.000000000' '2023-10-22T21:39:35.000000000'
 '2023-10-22T21:39:36.000000000' '2023-10-22T21:39:37.000000000'
 '2023-10-22T21:39:38.000000000' '2023-10-22T21:39:39.000000000'
 '2023-10-22T21:39:40.000000000' '2023-10-22T21:39:41.000000000'
 '2023-10-22T21:39:42.000000000' '2023-10-22T21:39:43.000000000'
 '2023-10-22T21:39:44.000000000']>

In [3]:
# rs in IETF
errors, rs, vs = satellite.sgp4_array(times.jd1, times.jd2)
assert (errors == 0).all()

In [4]:
from astropy.coordinates import SkyCoord, ITRS, CartesianRepresentation, CartesianDifferential, TEME, GCRS, AltAz, WGS84GeodeticRepresentation

cart_teme = CartesianRepresentation(
        rs << u.km,
        xyz_axis=-1,
        differentials=CartesianDifferential(
            vs << (u.km / u.s),
            xyz_axis=-1,
        ),
    )

# creates an ITRF
skycoord = SkyCoord(TEME(cart_teme, obstime=times), frame="teme", obstime=times).transform_to("itrs")
skycoord.representation_type = WGS84GeodeticRepresentation
skycoord

<SkyCoord (ITRS: obstime=['2023-10-22T21:39:34.000000000' '2023-10-22T21:39:35.000000000'
 '2023-10-22T21:39:36.000000000' '2023-10-22T21:39:37.000000000'
 '2023-10-22T21:39:38.000000000' '2023-10-22T21:39:39.000000000'
 '2023-10-22T21:39:40.000000000' '2023-10-22T21:39:41.000000000'
 '2023-10-22T21:39:42.000000000' '2023-10-22T21:39:43.000000000'
 '2023-10-22T21:39:44.000000000'], location=(0., 0., 0.) km): (lon, lat, height) in (deg, deg, m)
    [(59.60338587, -75.75864174, 806563.25424452),
     (59.66019811, -75.70132746, 806552.29358205),
     (59.71653377, -75.64399739, 806541.28880115),
     (59.77239839, -75.5866517 , 806530.23994174),
     (59.82779747, -75.52929058, 806519.1470439 ),
     (59.88273639, -75.47191421, 806508.01014784),
     (59.93722047, -75.41452276, 806496.82929398),
     (59.99125493, -75.35711639, 806485.60452285),
     (60.04484494, -75.29969529, 806474.33587516),
     (60.09799557, -75.24225961, 806463.02339179),
     (60.15071182, -75.18480951, 806451.66

In [149]:
skycoord.frame.cache

defaultdict(dict, {})

In [134]:
skycoord.frame.cache["representation"][('WGS84GeodeticRepresentation', True)][0].lon

<Longitude 59.60338587 deg>

In [106]:
skycoord.frame._data_repr().replace("[","").replace("]","").replace("(","").replace(")","").split("\n")[1:]

['    59.60338587, -75.75864174, 806563.25424452,',
 '     59.66019811, -75.70132746, 806552.29358205,',
 '     59.71653377, -75.64399739, 806541.28880115,',
 '     59.77239839, -75.5866517 , 806530.23994174,',
 '     59.82779747, -75.52929058, 806519.1470439 ,',
 '     59.88273639, -75.47191421, 806508.01014784,',
 '     59.93722047, -75.41452276, 806496.82929398,',
 '     59.99125493, -75.35711639, 806485.60452285,',
 '     60.04484494, -75.29969529, 806474.33587516,',
 '     60.09799557, -75.24225961, 806463.02339179,',
 '     60.15071182, -75.18480951, 806451.66711373']